# ParaTransit Analysis

---

1. Import packages
2. Loading data with Pandas
3. Descriptive statistics of data
4. Hypothesis investigation

---

## 1. Import packages

In [1]:
import pandas as pd

---

## 2. Loading data with Pandas

In [2]:
trip_df = pd.read_csv('./trips.csv')

In [3]:
trip_df.head()

,TripId,Clientid,Status,ProviderType,Outcome,Tripdate,RequestTime,PromiseTime,ProviderId,APtime,APtime1,ADtime,Anchor,PickCounty,PickZip,DropCounty,DropZip,TripMiles
0,1,18958,CX,E-Hail,Early Cancel,2022-04-01,06:30,05:35,NaN,00:00,00:00,00:00,P,Kings County,11228,Kings County,11219,0.00
1,2,33563,S,E-Hail,Authorized,2022-04-01,23:00,22:03,ISA,00:00,00:00,00:00,A,Richmond County,10308,Richmond County,10307,7.63
2,9,51426,CX,E-Hail,Early Cancel,2022-04-01,15:30,14:48,NaN,00:00,00:00,00:00,P,New York County,10022,Richmond County,10301,0.00
3,20,37638,CX,E-Hail,Early Cancel,2022-04-01,14:30,14:30,NaN,00:00,00:00,00:00,P,Kings,11209,Richmond,10314,0.00
4,21,36285,S,Primary,Completed,2022-04-01,09:30,09:30,MAG,09:38,00:00,09:55,P,Kings,11203,Kings,11226,0.90


We have a mix of numeric and categorical data.

---

## 3. Descriptive statistics of data

### Data types

In [4]:
trip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1164013 entries, 0 to 1164012
Data columns (total 18 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   TripId        1164013 non-null  int64  
 1   Clientid      1164013 non-null  int64  
 2   Status        1164013 non-null  object 
 3   ProviderType  1164013 non-null  object 
 4   Outcome       1164013 non-null  object 
 5   Tripdate      1164013 non-null  object 
 6   RequestTime   1164013 non-null  object 
 7   PromiseTime   1164013 non-null  object 
 8   ProviderId    543476 non-null   object 
 9   APtime        1164012 non-null  object 
 10  APtime1       1164013 non-null  object 
 11  ADtime        1164012 non-null  object 
 12  Anchor        1164013 non-null  object 
 13  PickCounty    1138339 non-null  object 
 14  PickZip       1138336 non-null  object 
 15  DropCounty    1138338 non-null  object 
 16  DropZip       1138336 non-null  object 
 17  TripMiles     1164013 non-n

All of the `datetime` related columns are not currently in datetime format. We will need to convert these later.

### Statistics

In [5]:
trip_df.describe()

,TripId,Clientid,TripMiles
count,1.164013e+06,1.164013e+06,1.164013e+06
mean,5.820057e+05,2.911862e+04,3.362872e+00
std,3.360214e+05,1.716323e+04,5.427824e+00
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.910030e+05,1.423400e+04,0.000000e+00
50%,5.820060e+05,2.958800e+04,0.000000e+00
75%,8.730080e+05,4.385000e+04,5.130000e+00
max,1.164011e+06,5.931200e+04,4.649000e+01


Ignore the `TripID` and `ClientID`'s descriptions since they are not exactly 'numerical' variables. Overall the data looks good. The maximum of `TripMiles` is 46 miles, which might be potential outlier. We can proceed with caution.

---

## 4. Hypothesis Analysis

It's time to investigate the cost efficiency of the Paratransit service among the 3 types of services - Primary, Broker, and E-Hail by look into the proportion of successfully completed trips during weekdays and weekends.

In [6]:
#Checking the date range

print('Minimum Date: ', min(trip_df.Tripdate))
print('Minimum Date: ', max(trip_df.Tripdate))

Minimum Date:  2022-04-01
Minimum Date:  2022-04-30


This data is for April 2022 only.

Since we are manipulating the `Tripdate` column, we need to convert the column's datatype to `datetime`.

In [7]:
trip_df['Tripdate']=pd.to_datetime(trip_df['Tripdate'])
trip_df['Tripdate'].dtypes

dtype('<M8[ns]')

    > My approach is to: 
        1. Create several subsets of data from the main dataframe
        2. Perform the calculation and store it in lists
        3. Convert the lists to dataframe tables

In [8]:
#Lists of categorical values of successful trips among the 3 types of service
provider_type=['Primary','Broker','E-Hail']
trip_outcome=['Authorized','Completed']

#Making subsets of completed trips during weekdays and weekend 
weekday_successful_df=trip_df.loc[(trip_df['Outcome'].isin(trip_outcome)) & (trip_df['Tripdate'].dt.weekday<5)]
weekend_successful_df=trip_df.loc[(trip_df['Outcome'].isin(trip_outcome)) & (trip_df['Tripdate'].dt.weekday>=5)]

 >**Total successful trips**

In [9]:
#Empty lists for total trips values for weekdays and weekends
weekday_successful=[]
weekend_successful=[]

#Loop through the types of service and append the count of records for weekdays and weekends from the above subsets to the above empty lists
for i in provider_type:
    weekday_successful.append(len(weekday_successful_df[(weekday_successful_df['ProviderType']==i)]))
    weekend_successful.append(len(weekend_successful_df[(weekend_successful_df['ProviderType']==i)]))

#Calculate the total trips for weekday and weekend and append it to the list
total_weekday_successful=sum(weekday_successful)
total_weekend_successful=sum(weekend_successful)
weekday_successful.append(total_weekday_successful)
weekend_successful.append(total_weekend_successful)

#Total trips for each service type
total_successful_mode=[x + y for x, y in zip(weekday_successful, weekend_successful)]

In [10]:
#Header and row index for dataframe transformation
header=['Weekday','Weekend','Total']
row_header=['Primary','Broker','E-Hail','Total']

#Convert the 3 lists to dataframe, using the header and row index
total_successful=list(zip(weekday_successful,weekend_successful,total_successful_mode))
total_successful_df=pd.DataFrame(total_successful,index=row_header,columns=header)

#Tranpose the dataframe
total_successful_df=total_successful_df.T

In [11]:
total_successful_df

,Primary,Broker,E-Hail,Total
Weekday,112506,230605,86357,429468
Weekend,27153,58012,17483,102648
Total,139659,288617,103840,532116


From this table we can see the number of successful weekend trips is generally 20-25% of that of the weekday, for all 3 modes. Regarding the concern between the primary and broker mode, primary has fewer successful trips, accounting of around half as much as broker. The total of successful broker trip is more than half of the grand total. 

 >**Average trips per day**

In [12]:
#Calculate the number of weekdays and weekends to for average calculations
weekday_count=len(pd.date_range(min(trip_df.Tripdate),max(trip_df.Tripdate),freq='B'))
weekend_count=len(pd.date_range(min(trip_df.Tripdate),max(trip_df.Tripdate),freq='D'))-weekday_count

print('# of weekdays: ', weekday_count)
print('# of weekends: ',weekend_count)

# of weekdays:  21
# of weekends:  9


In [13]:
#Empty list to store average values for weekdays and weekends
avg_weekday=[]
avg_weekend=[]

#Loop through the weekday/weekend total trips list and divide by the number of weekdays/weekends, round to 2 decimal places
for i in weekday_successful:
    avg_weekday.append(round(i/weekday_count,2))
for j in weekend_successful:
    avg_weekend.append(round(j/weekend_count,2))

In [14]:
#Convert the 2 lists to dataframe, using the header and slicing of row index
total_average=list(zip(avg_weekday,avg_weekend))
total_average_df=pd.DataFrame(total_average,index=row_header,columns=header[:2])

#Tranpose the dataframe
total_average_df=total_average_df.T

In [15]:
total_average_df

,Primary,Broker,E-Hail,Total
Weekday,5357.43,10981.19,4112.24,20450.86
Weekend,3017.00,6445.78,1942.56,11405.33


The average successful trips is approximately halved from the total number of successful trips between weekday and weekend, since there are more than twice as much as the number of weekdays than weekends. Successful primary trips are still half as much as broker's, which is also still half of the total.

 >**Percent of trips by mode**

In [16]:
#Empty list to store percentage values for weekdays and weekends
weekday_percent=[]
weekend_percent=[]

# Loop through the weekday/weekend total trips list and divide by the total trips of weekdays/weekends, multiply by 100, and round to 2 decimal places
for i in weekday_successful:
    weekday_percent.append(str(round((i/total_weekday_successful)*100,2))+"%")
for j in weekend_successful:
    weekend_percent.append(str(round((j/total_weekend_successful)*100,2))+"%")

In [17]:
#Convert the 2 lists to dataframe, using the header and slicing of row index
total_percentage=list(zip(weekday_percent,weekend_percent))
total_percentage_df=pd.DataFrame(total_percentage,index=row_header,columns=header[:2])

#Tranpose the dataframe
total_percentage_df=total_percentage_df.T

In [18]:
total_percentage_df

,Primary,Broker,E-Hail,Total
Weekday,26.2%,53.7%,20.11%,100.0%
Weekend,26.45%,56.52%,17.03%,100.0%


The weekday and weekend percentage for all 3 modes are relatively the same. It also confirms the accuracy of our analysis above - primary has half as much trips as broker, and one-third of broker and e-hail combined. Broker and e-hail trips are 75% of all successful trips. Considering the fact that primary trips are more expensive than broker and e-hail, this analysis shows a positive outcome, especially when the MTA receive less information and detail about the broker and e-hail trips.

 >**Top 10 zip codes where Primary trip pickups take place during the AM rush hour (6-10 AM)** 
 
Now that we have the proportion of trips completed on Primary versus other mode, it is viable to make the Paratransit service more cost-effective by starting a new circular procurement for more providers. By showing the top 10 zip codes that is most common for primary trip pickups, this new 'geographic score' feature might be helpful to appropriately distribute the providers from depots to the pickups, thus saving cost and time for both the MTA and the customers.

Since the problem asks for the scheduled time to pickup, `PromiseTime` is the best fit in this case. As we have some timestamps larger than 23:59, let's check the maximum time that we have in the entire dataset.

In [19]:
print('Maximum time: ', max(trip_df['PromiseTime']))
print('New maximum hour: ',int(max(trip_df['PromiseTime'].str.slice(stop=2)))-24)

Maximum time:  29:50
New maximum hour:  5


The largest timestamp that expanded in a day after is still earlier than 6 AM, so we can temporarily ignore those records for the purpose of this problem. There is a potentiality of missing records but it should be very minimal.

In [20]:
#A subset of successful primary trip
successful_primary=trip_df.loc[(trip_df['Outcome']=='Completed') & (trip_df['ProviderType']=='Primary')]

In [21]:
#A subset of scheduled (promise) pickup during AM rush hour for successful primary trip
successful_primary_promised=successful_primary.loc[(successful_primary['PromiseTime']>='06:00') & (successful_primary['PromiseTime']<='10:00')]
successful_primary_promised.head()


,TripId,Clientid,Status,ProviderType,Outcome,Tripdate,RequestTime,PromiseTime,ProviderId,APtime,APtime1,ADtime,Anchor,PickCounty,PickZip,DropCounty,DropZip,TripMiles
4,21,36285,S,Primary,Completed,2022-04-01,09:30,09:30,MAG,09:38,00:00,09:55,P,Kings,11203,Kings,11226,0.90
8,56,4798,S,Primary,Completed,2022-04-01,07:15,07:15,MVT,07:20,07:14,07:32,P,Kings,11221,Kings,11213,1.13
9,70,51875,S,Primary,Completed,2022-04-01,09:45,09:45,ITA,10:01,09:57,11:03,P,Queens,11429,New York,10031,19.93
11,88,21318,S,Primary,Completed,2022-04-01,07:30,07:30,MVT,07:31,07:30,09:01,P,Kings,11236,Kings,11217,7.19
20,156,36868,S,Primary,Completed,2022-04-01,10:00,10:00,PTC,10:05,09:56,10:51,P,New York,10016,Kings,11206,5.28


In [22]:
#Group by zip codes, find the first 10 zips based on the number of trips, ascendingly; and convert to dataframe
top_10_zipcode=((successful_primary_promised.PickZip.value_counts()).head(10)).to_frame()

#Format the table resetting the index column names
top_10_zipcode.reset_index(inplace=True)
top_10_zipcode.columns=['ZIP Code','Number of Primary Trips During AM Rush Hour']
top_10_zipcode.index+=1

In [23]:
top_10_zipcode

,ZIP Code,Number of Primary Trips During AM Rush Hour
1,10314,1220
2,11212,905
3,10304,898
4,11207,859
5,10301,792
6,10306,781
7,11434,732
8,11233,664
9,11691,646
10,11236,614


In [24]:
#Calculate total number of trips in rush hour from the 10 zip codes and percentage
total_top_10 = top_10_zipcode['Number of Primary Trips During AM Rush Hour'].sum()
print(round((total_top_10/len(successful_primary))*100,2),"%")

5.81 %


The total number of trips scheduled for pickups in the top 10 zip codes using primary service only accounted for around 6%, which is not that significant. However, it is still useful to include this feature in the cost proposal, if the MTA is looking to expand this feature into other timeframes (afternoon rush hour for example) or modes as it still helps the MTA saving on cost in the long run.

 >**Mean trip distance and total number of intra/interboro trips within the 4 zip codes and/or Manhattan** 
 
Now we are looking into the implementation of 'B-Leg Flexibility' for the customers who wants to be flexible to request the return leg on shorter notice to see if it is cost-feasible, by looking at the mean trip distance and the total number of trips starting/ending in the 4 zip codes of 11201, 10469, 11694, 10306 and/or Manhattan. We will be able to see the general trend of cost to serve Paratransit customers based on their trip location after this analysis.

In [25]:
#List of zip codes and county involved in the problem and their pairs
zip_codes=['11201','10469','11694','10306']
county=['Kings','Bronx','Queens','Richmond']
zip_county_pair=list(zip(zip_codes,county))

#Subset of successful trips
successful_trip_df=trip_df.loc[(trip_df['Outcome'].isin(trip_outcome))]

    > Create subsets of data that satisfies the pickup/dropoff for intra/inter-boro trips and total by: 
        1. For each trip type:
            a. Creating an empty dictionary
            b. Creating empty lists to store the values of mean distance and total number of trips
        3. Storing the data subsets in different variables (dataframes) when looping through each zip code (and/or county) 
        4. Indicing through the dictionary and do the calculation for each dataframe

In [26]:
intra_dict={}
intra_total_trip=[]
intra_mean_distance=[]

for i in zip_county_pair:
    #pickup from zip code - dropoff at same borough OR pickup from same borough - dropoff at zip code
    intra_dict["intra_{0}".format(i[0])]=pd.concat([successful_trip_df.loc[(successful_trip_df['PickZip']==i[0]) & (successful_trip_df['DropCounty']==i[1])],
                                                     successful_trip_df.loc[(successful_trip_df['DropZip']==i[0]) & (successful_trip_df['PickCounty']==i[1])]])

for k,v in intra_dict.items():
    intra_total_trip.append(len(intra_dict[k]))
    intra_mean_distance.append(round((intra_dict[k]['TripMiles'].sum())/len(intra_dict[k]),2))

In [27]:
inter_dict={}
inter_total_trip=[]
inter_mean_distance=[]

for i in zip_codes:
    #pickup from zip code - dropoff at Manhattan OR pickup from Manhattan - dropoff at zip code
    inter_dict["inter_{0}".format(i)]=pd.concat([successful_trip_df.loc[(successful_trip_df['PickZip']==i) & (successful_trip_df['DropCounty']=='New York')],
                                        successful_trip_df.loc[(successful_trip_df['DropZip']==i) & (successful_trip_df['PickCounty']=='New York')]])
    
for k,v in inter_dict.items():
    inter_mean_distance.append(round(inter_dict[k]['TripMiles'].sum()/len(inter_dict[k]),2))
    inter_total_trip.append(len(inter_dict[k]))

In [28]:
total_dict={}
total_mean_distance=[]
total_trip=[]

for i in zip_codes:
    #pick up OR dropoff at zip code
    total_dict["total_{0}".format(i)]=successful_trip_df.loc[(successful_trip_df['PickZip']==i) | (successful_trip_df['DropZip']==i)]

for k,v in total_dict.items():
    total_mean_distance.append(round(total_dict[k]['TripMiles'].sum()/len(total_dict[k]),2))
    total_trip.append(len(total_dict[k]))

In [29]:
header=['Intra-boro Mean Distance', 'Intra-boro Total Trips', 'Inter-boro Mean Distance', 'Inter-boro Total Trips', 'Total Mean Distance', 'Total Trips']
flex_b_leg=list(zip(intra_mean_distance,intra_total_trip,inter_mean_distance,inter_total_trip,total_mean_distance,total_trip))
flex_b_leg_df=pd.DataFrame(flex_b_leg,index=zip_codes,columns=header)
flex_b_leg_df

,Intra-boro Mean Distance,Intra-boro Total Trips,Inter-boro Mean Distance,Inter-boro Total Trips,Total Mean Distance,Total Trips
11201,4.34,11850,6.50,1911,6.69,18109
10469,2.91,4815,10.78,2118,7.12,8101
11694,7.85,940,19.29,284,10.95,1933
10306,3.87,4414,20.57,676,8.54,6213


Based on the 4 ZIP codes, there are 2 aspects to consider. It is the distance of the trips, and the number of newly requested trips within a shorter period of time. Regarding the distance, it is apparent that further zip codes will incur more expenses (Queens and Staten Island in the above table). In respect to the latter, not only the number of miles matters, but also the cost to distribute the provider from the depot/their current location to the pickup location of the customers (that we did a brief analysis on Problem 2) is significant (which might potentially be Brooklyn and the Bronx given their higher total number of trips, specifically when the distances from/to these boroughs are shorter so there might be more flexibility for late scheduling).

## What I could have done differently
- The code could be neater where I can utilize more loops for data subsetting, dataframe conversions, and some repetitive calculations, especially for question 1 and 3, which would help tremendously in the automation process and bug fixing in the future.
- For question 2, I could parse the PromiseTime hour and minute and do the transformation to the next day's time to retain the generalization of the data.
